## Import Required Libraries

We'll import all necessary Python libraries for financial data analysis, portfolio optimization, and visualization.

In [2]:
import warnings
import os
from datetime import datetime, timedelta
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyoff
from plotly.subplots import make_subplots
from IPython.display import display, Markdown

from Backtester.BacktestResults import TestResults

warnings.filterwarnings('ignore')

## Load backtest data

In [3]:
universe_name = "selection3";test_name = "GEM_LP"

test_path = f"data/{universe_name}/{test_name}"
test_results = TestResults(test_path)
strategies_list = test_results.list_strategies()
test_images_path = os.path.join(test_path, "images",test_name)
os.makedirs(test_images_path, exist_ok=True)

md = f"""
### Backtest Settings

- Test path: `{test_path}`
- Number of strategies: **{len(strategies_list)}**

```json
{json.dumps(test_results.test_settings, indent=2)}
```
### Strategies
```json
{json.dumps(strategies_list, indent=2)}
```
"""
display(Markdown(md))


### Backtest Settings

- Test path: `data/selection3/GEM_LP`
- Number of strategies: **12**

```json
{
  "universe_name": "selection3",
  "backtest_duration": 504,
  "lookback_periods": 127,
  "num_datasets": 100,
  "random_seed": 12,
  "num_assets": null,
  "test_name": "GEM_LP",
  "test_folder_path": "c:\\my-git\\DataScience-novaIMS\\APPM-individual\\data\\selection3\\GEM_LP"
}
```
### Strategies
```json
[
  "AAA1",
  "AAA2",
  "AAA3",
  "AAA4",
  "AAA_M1",
  "EqualWeight",
  "GEM1",
  "GEM2",
  "GEM3",
  "GEM4",
  "GEM5",
  "GEM_M1"
]
```


In [18]:
analyze_strategy = "AAA1"
dataset_name="dataset_35"
#analyze_by = "VWR"
#top_and_bottom = 5

In [19]:
strategy_images_path = os.path.join(test_images_path, analyze_strategy)
os.makedirs(strategy_images_path, exist_ok=True)

strategy_returns = test_results.strategies[analyze_strategy].get_datasets_returns(dataset_names=[dataset_name])
strategy_cum_returns = (1 + strategy_returns).cumprod()

datasets_info = test_results.datasets
dataset_data = test_results.get_datasets_data(datasets_filter_list=[dataset_name],column="adjusted")
portfolio_data = test_results.strategies[analyze_strategy].datasets[dataset_name].get_asset_values('portfolio')
portfolio_value_history = portfolio_data['portfolio_value']
dataset_adjusted = pd.DataFrame(dataset_data.get(dataset_name)).pct_change().cumsum() + 1
start_date = datasets_info[dataset_name]['start_date']
end_date = datasets_info[dataset_name]['end_date']

datasets_weights = test_results.strategies[analyze_strategy].get_datasets_weights()
wdf = datasets_weights.get(dataset_name)
asset_value_history = pd.DataFrame(test_results.strategies[analyze_strategy].datasets[dataset_name].get_asset_values('values'))
asset_value_history['Cash'] = portfolio_data['cash']

# Create subplots
fig = make_subplots(
    rows=3, cols=1,
    specs=[[{"type": "scatter"}], [{"type": "bar"}], [{"type": "bar"}]],
    vertical_spacing=0.08,
    row_heights=[0.5, 0.25, 0.25]
)

colors = px.colors.qualitative.Plotly

# Subplot 1: Cumulative Returns
x_dates = dataset_adjusted.index
sorted_cols = sorted(dataset_adjusted.columns)
ncols_total = len(sorted_cols)
color_cycle_main = list(np.tile(colors, int(np.ceil(ncols_total / len(colors)))))[:ncols_total]

for i, col in enumerate(sorted_cols):
    fig.add_trace(go.Scatter(
        x=x_dates,
        y=dataset_adjusted[col],
        mode='lines',
        name=col,
        line=dict(width=1, color=color_cycle_main[i]),
        hovertemplate=f"{col}<br>Date: %{{x}}<br>Cumulative Return: %{{y:.4f}}<extra></extra>",
        legendgroup=col,
        showlegend=True
    ), row=1, col=1)

#fig.add_trace(go.Scatter(
#    x=portfolio_value_history.index,
#    y=portfolio_value_history.values / portfolio_value_history.values[0],
#    mode='lines',
#    name='Portfolio Value',
#    line=dict(color='red', width=2),
#    hovertemplate="Portfolio Value<br>Date: %{x}<br>Normalized Value: %{y:.4f}<extra></extra>",
#    legendgroup='Portfolio Value',
#    showlegend=True
#), row=1, col=1)
fig.add_trace(go.Scatter(
    x=x_dates,
    y=strategy_cum_returns[dataset_name],
    mode='lines',
    name='Total',
    line=dict(color='black', width=2),
    hovertemplate="Total<br>Date: %{x}<br>Cumulative Return: %{y:.4f}<extra></extra>"
))
# Subplot 2: Weights
if wdf is not None and not wdf.empty:
    sorted_weight_cols = sorted(wdf.columns)
    
    for col_name in sorted_weight_cols:
        # Find matching color from sorted_cols
        if col_name in sorted_cols:
            col_idx = sorted_cols.index(col_name)
            bar_color = color_cycle_main[col_idx]
        else:
            bar_color = 'gray'
        
        fig.add_trace(go.Bar(
            x=wdf.index,
            y=wdf[col_name],
            name=col_name,
            marker_color=bar_color,
            hovertemplate=f"{col_name}<br>Date: %{{x}}<br>Weight: %{{y:.6f}}<extra></extra>",
            legendgroup=col_name,
            showlegend=False
        ), row=2, col=1)

# Subplot 3: Asset Values
sorted_asset_cols = sorted(asset_value_history.columns)

for col_name in sorted_asset_cols:
    # Find matching color from sorted_cols or use special color for Cash
    if col_name == 'Cash':
        bar_color = 'lightgray'
        legend_group = 'Cash'
    elif col_name in sorted_cols:
        col_idx = sorted_cols.index(col_name)
        bar_color = color_cycle_main[col_idx]
        legend_group = col_name
    else:
        bar_color = 'gray'
        legend_group = col_name
    
    fig.add_trace(go.Bar(
        x=asset_value_history.index,
        y=asset_value_history[col_name],
        name=col_name,
        marker_color=bar_color,
        hovertemplate=f"{col_name}<br>Date: %{{x}}<br>Value: %{{y:.2f}}<extra></extra>",
        legendgroup=legend_group,
        showlegend=False
    ), row=3, col=1)

fig.update_xaxes(title_text='Date', row=1, col=1)
fig.update_xaxes(title_text='Date', row=2, col=1)
fig.update_xaxes(title_text='Date', row=3, col=1)
fig.update_yaxes(title_text='Cumulative Return', row=1, col=1)
fig.update_yaxes(title_text='Weight', range=[0, 1], row=2, col=1)
fig.update_yaxes(title_text='Asset Value ($)', row=3, col=1)

fig.update_layout(
    barmode='stack',
    height=1300,
    showlegend=True,
    legend=dict(
        orientation="v", 
        yanchor="top", 
        y=1, 
        xanchor="left", 
        x=1.02,
        itemsizing='constant',
        tracegroupgap=0
    )
)

pyoff.iplot(fig)


In [20]:
# Calculate Drawdown

running_max = strategy_cum_returns[dataset_name].cummax()
drawdown = (strategy_cum_returns[dataset_name] - running_max) / running_max
drawdown.index = pd.to_datetime(x_dates)
# Create drawdown chart
fig_dd = go.Figure()

# Add drawdown area
fig_dd.add_trace(go.Scatter(
    x=drawdown.index,
    y=drawdown * 100,  # Convert to percentage
    fill='tozeroy',
    name='Drawdown',
    line=dict(color='red', width=1),
    fillcolor='rgba(255, 0, 0, 0.3)',
    hovertemplate='Date: %{x}<br>Drawdown: %{y:.2f}%<extra></extra>'
))

# Add zero line
fig_dd.add_hline(y=0, line_dash="dash", line_color="gray", line_width=1)

# Calculate key drawdown statistics
max_drawdown = drawdown.min() * 100
max_dd_date = drawdown.idxmin()
current_drawdown = drawdown.iloc[-1] * 100

# Add annotations for max drawdown
fig_dd.add_annotation(
    x=max_dd_date,
    y=max_drawdown,
    text=f'Max DD: {max_drawdown:.2f}%',
    showarrow=True,
    arrowhead=2,
    arrowcolor='red',
    ax=0,
    ay=-40
)

fig_dd.update_layout(
    title=f'Drawdown Chart - {analyze_strategy} - {dataset_name}',
    xaxis_title='Date',
    yaxis_title='Drawdown (%)',
    height=500,
    showlegend=True,
    hovermode='x unified'
)

# Display statistics
display(Markdown(f"""
### Drawdown Statistics for {dataset_name}
- **Maximum Drawdown:** {max_drawdown:.2f}%
- **Max Drawdown Date:** {max_dd_date.strftime('%Y-%m-%d')}
- **Current Drawdown:** {current_drawdown:.2f}%
- **Average Drawdown:** {(drawdown[drawdown < 0].mean() * 100):.2f}%
"""))

pyoff.iplot(fig_dd)


### Drawdown Statistics for dataset_35
- **Maximum Drawdown:** -14.40%
- **Max Drawdown Date:** 2020-03-18
- **Current Drawdown:** -2.41%
- **Average Drawdown:** -4.18%


In [ ]:
order_history = test_results.strategies[analyze_strategy].datasets[dataset_name].get_orders()

total_comm=order_history['commission'].sum()
avg_pnl=order_history['pnl'].mean()
display(Markdown(f"**Total Commissions for {dataset_name}:** ${total_comm:,.2f}"))
display(Markdown(f"**Average PnL for {dataset_name}:** ${avg_pnl:,.2f}"))
#display total number of orders and average size
total_orders = order_history.shape[0]
sell_orders = order_history[order_history['order_type'] == 'SELL']
buy_orders = order_history[order_history['order_type'] == 'BUY']


# calculate and display profit loss ratio
# Filter winning and losing trades (based on sell orders which realize P&L)
winning_trades = sell_orders[sell_orders['pnl'] > 0]
losing_trades = sell_orders[sell_orders['pnl'] < 0]

if len(winning_trades) > 0 and len(losing_trades) > 0:
    avg_win = winning_trades['pnl'].mean()
    avg_loss = abs(losing_trades['pnl'].mean())
    profit_loss_ratio = avg_win / avg_loss
    
    win_rate = len(winning_trades) / len(sell_orders) * 100
    loss_rate = len(losing_trades) / len(sell_orders) * 100
    
    display(Markdown(f"### Profit/Loss Ratio Analysis for {dataset_name}"))
    display(Markdown(f"- **Profit/Loss Ratio:** {profit_loss_ratio:.2f}:1"))
    display(Markdown(f"- **Average Winning Trade:** ${avg_win:,.2f}"))
    display(Markdown(f"- **Average Losing Trade:** ${avg_loss:,.2f}"))
    display(Markdown(f"- **Win Rate:** {win_rate:.1f}% ({len(winning_trades)} trades)"))
    display(Markdown(f"- **Loss Rate:** {loss_rate:.1f}% ({len(losing_trades)} trades)"))
    
    # Calculate APPT (Average Profitability Per Trade)
    appt = (win_rate/100 * avg_win) - (loss_rate/100 * avg_loss)
    display(Markdown(f"- **APPT (Average Profitability Per Trade):** ${appt:,.2f}"))
else:
    display(Markdown(f"**Note:** Insufficient data to calculate Profit/Loss Ratio"))

buy_orders = order_history[order_history['order_type'] == 'BUY']
sell_orders = order_history[order_history['order_type'] == 'SELL']
avg_order_size = buy_orders['executed_size'].mean()
display(Markdown(f"**Total Orders for {dataset_name}:** {total_orders}"))
display(Markdown(f"**Average Order Size for {dataset_name}:** {avg_order_size:,.2f}"))
# from sell_orders display the average pnl per asset
avg_pnl_sell = sell_orders.groupby('asset')['pnl'].sum()
avg_pnl_sell.sort_values(ascending=False, inplace=True)
display(Markdown(f"**Average PnL per Asset for {dataset_name}:**"))
display(avg_pnl_sell)

#plot an histogram of the order PnL
fig_pnl = px.histogram(order_history, x='pnl', nbins=50, title=f'Order PnL Distribution for {dataset_name}',
                       labels={'pnl': 'Profit and Loss ($)'})
fig_pnl.update_layout(height=500)
pyoff.iplot(fig_pnl)

#plot an histogram of the order PnL
fig_pnl = px.histogram(buy_orders, x='executed_size', nbins=50, title=f'Order Executed Size Distribution for {dataset_name}',
                       labels={'executed_size': 'Executed Size'})
fig_pnl.update_layout(height=500)
pyoff.iplot(fig_pnl)

**Total Commissions for dataset_35:** $3,060.04

**Average PnL for dataset_35:** $29.64

NameError: name 'sell_orders' is not defined

In [ ]:
display(pd.DataFrame(portfolio_value_history))
display(pd.DataFrame(order_history).sort_values('pnl', ascending=True).head(10))
display(pd.DataFrame(order_history).sort_values('pnl', ascending=False).head(10))

,portfolio_value
date,
2020-03-02,100000.000000
2020-03-31,87865.039655
2020-04-30,87929.243809
2020-06-01,88611.391049
2020-06-30,89930.557696
2020-07-30,93379.744523
2020-08-28,96114.788487
2020-09-29,92268.859626
2020-10-28,90695.295312


,asset,order_type,status,created_date,created_price,created_size,executed_date,executed_price,executed_size,executed_value,commission,pnl
order_id,,,,,,,,,,,,
13,EWW,SELL,Completed,NoneType,28.350000,-225.0,2020-04-01,27.049999,-225.0,9639.000034,6.08625,-3552.750206
16,CANE,SELL,Completed,NoneType,5.460000,-1369.0,2020-04-01,5.400000,-1369.0,9610.379974,7.39260,-2217.779843
11,EWD,SELL,Completed,NoneType,25.590000,-313.0,2020-04-01,24.620001,-313.0,9853.239857,7.70606,-2147.179594
18,EWY,SELL,Completed,NoneType,46.919998,-172.0,2020-04-01,44.419998,-172.0,9618.239685,7.64024,-1978.000000
12,EWT,SELL,Completed,NoneType,32.970001,-250.0,2020-04-01,32.349998,-250.0,9715.000153,8.08750,-1627.500534
168,TUR,SELL,Completed,NoneType,22.990000,-393.0,2021-04-01,23.700001,-393.0,10878.240120,9.31410,-1564.139820
17,AIA,SELL,Completed,NoneType,55.709999,-155.0,2020-04-01,54.610001,-155.0,9740.200024,8.46455,-1275.649929
180,ECH,SELL,Completed,NoneType,31.900000,-399.0,2021-05-03,32.020000,-399.0,13773.480183,12.77598,-997.500000
134,GREK,SELL,Completed,NoneType,23.200001,-390.0,2021-02-01,23.450001,-390.0,10073.699970,9.14550,-928.199673


,asset,order_type,status,created_date,created_price,created_size,executed_date,executed_price,executed_size,executed_value,commission,pnl
order_id,,,,,,,,,,,,
164,UCO,SELL,Completed,NoneType,13.212500,-839.0,2021-04-01,13.842500,-839.0,8790.622468,11.613857,2823.234808
215,UCO,SELL,Completed,NoneType,19.977501,-611.0,2021-08-02,19.762501,-611.0,9253.595280,12.074888,2821.292686
195,EWY,SELL,Completed,NoneType,93.459999,-125.0,2021-06-02,92.879997,-125.0,8811.477939,11.610000,2798.521718
105,QQQ,SELL,Completed,NoneType,299.010010,-39.0,2020-11-30,299.339996,-39.0,8917.560013,11.674260,2756.699844
106,ARGT,SELL,Completed,NoneType,28.660000,-467.0,2020-11-30,28.389999,-467.0,10844.369680,13.258130,2413.760035
191,EWT,SELL,Completed,NoneType,63.930000,-175.0,2021-06-02,63.639999,-175.0,9075.590801,11.137000,2061.409092
135,CPER,SELL,Completed,NoneType,21.910000,-467.0,2021-02-01,21.980000,-467.0,8568.632387,10.264660,1696.027399
153,INDA,SELL,Completed,NoneType,42.439999,-259.0,2021-03-03,43.020000,-259.0,9533.639687,11.142180,1608.540432
82,GLD,SELL,Completed,NoneType,178.190002,-50.0,2020-09-30,177.710007,-50.0,7446.589844,8.885500,1438.910492
